In [1]:
#If the code is running on Google Colab, I need to mount the GDrive into the workspace to import my homemade dependencies
google_colab = True
if google_colab:
    #Mount google drive into google colab working space
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    #Copy my functions into Colab workspace
    !cp "/content/drive/MyDrive/Colab Notebooks/Federated Learning/models.py" .

Mounted at /content/drive


In [2]:
#Imports
import models
import numpy as np
import time
import tensorflow as tf
from matplotlib import pyplot as plt

In [3]:
#Simulation Variables
K = 2
local_epochs = 1
steps_per_epoch = None #None = iterate until entire dataset is covered once

In [4]:
#Download global dataset and split it into K local datasets
local_datasets, global_dataset = models.load_dataset(K)
#CIFAR10 labels
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
#Create K local models and one global model with randomly initialized weights
model_list, global_model = models.setup_models(K, len(labels))

170498071/170498071 [==============================] - 2s 0us/step


In [5]:
#Broadcast to devices
model_list, global_model = models.model_broadcast(model_list, global_model)

In [6]:
#Local training
start = time.time()
model_list = models.local_training(model_list, local_datasets, local_epochs, steps_per_epoch)
end = time.time()
print("Time spent training =", end-start)

391/391 [==============================] - 4s 8ms/step - loss: 1.6957 - accuracy: 0.3706
Time spent training = 16.385279417037964


In [7]:
#Global Aggregation
model_list, global_model = models.global_update(model_list, global_model)

In [8]:
#Accuracy evaluation 
results = global_model.evaluate(local_datasets[0][0], local_datasets[0][1], batch_size=128, verbose=0) 
print("train loss, train acc of global model:", results) 
results = global_model.evaluate(global_dataset[2], global_dataset[3], batch_size=128, verbose=0) 
print("test loss, test acc of global model:", results) 
results = model_list[0].evaluate(global_dataset[2], global_dataset[3], batch_size=128, verbose=0) 
print("test loss, test acc of local model 0:", results) 
results = model_list[0].evaluate(local_datasets[0][0], local_datasets[0][1], batch_size=128, verbose=0) 
print("train loss, train acc of local model 0:", results) 
results = model_list[1].evaluate(global_dataset[2], global_dataset[3], batch_size=128, verbose=0) 
print("test loss, test acc of local model 1:", results)

train loss, train acc of global model: [2.033526659011841, 0.3641600012779236]
test loss, test acc of global model: [2.0323848724365234, 0.36660000681877136]
test loss, test acc of local model 0: [1.3835734128952026, 0.4772000014781952]
train loss, train acc of local model 0: [1.3582721948623657, 0.49219998717308044]
test loss, test acc of local model 1: [1.4209994077682495, 0.4828000068664551]
